## Tools

First, let's set up our SQLite database connection using the Chinook sample database.


In [1]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Now we'll create our first tool: a SQL executor that can run queries against the database.


In [2]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

Next, we'll add a currency conversion tool to handle different currencies (the database stores prices in USD).


In [3]:
import { tool } from "langchain";

const FALLBACK_RATES = { USD: 1.0, EUR: 0.92, JPY: 150.0, GBP: 0.79 }

const convertCurrency = tool(async ({ amount, toCurrency }) => {
    try {
        const url = `https://api.exchangerate.host/convert?from=USD&to=${toCurrency}&amount=${amount}`
        const response = await fetch(url, {
            headers: {
                "Accept": "application/json"
            }
        })
        const data = await response.json()
        const result = data.result
        if (result) {
            return Math.round(amount * FALLBACK_RATES[to_currency], 2)
        }
    } catch (error) {
        console.error(`[convert_currency] Falling back due to error: ${error}`)
    }
    if (!(toCurrency in FALLBACK_RATES)) {
        throw new Error("Unsupported currency in fallback mode")
    }
    return Math.round(amount * FALLBACK_RATES[toCurrency], 2)
}, {
    name: "convert_currency",
    description: `
        Convert an amount in USD to another currency using live exchange rates.
        Always use this tool if the user requests a currency different 
        from the one stored in the database (USD).
    `,
    schema: z.object({
        amount: z.number(),
        toCurrency: z.string()
    })
});

Let's define the system prompt that will guide our agent's behavior as a SQL analyst.


In [4]:
const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
`

Now we can create our agent with both tools and the system prompt.


In [5]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [executeSQL, convertCurrency],
    systemPrompt: SYSTEM,
    middleware: []
})

Let's test the agent with a question that requires both SQL queries and currency conversion.


In [6]:
import { HumanMessage } from "langchain";

const question = "What is the most costly purchase by Frank Harris in EUR?";

const stream = await agent.stream({
    messages: [new HumanMessage(question)],
}, {
    streamMode: "values",
});

for await (const state of stream) {
    const message = state.messages.at(-1);
    const content = message.content || `Tool calls: ${JSON.stringify(message.tool_calls?.map(t => `${t.name}: ${JSON.stringify(t.args)}`).join(", "))}`;
    console.log(`${message.type} - ${content}`)
}

human - What is the most costly purchase by Frank Harris in EUR?


ai - Tool calls: "execute_sql: {\"query\":\"SELECT name FROM sqlite_master WHERE type='table' ORDER BY name LIMIT 5;\"}"
tool - [{"name":"Album"},{"name":"Artist"},{"name":"Customer"},{"name":"Employee"},{"name":"Genre"}]
ai - Tool calls: "execute_sql: {\"query\":\"SELECT i.InvoiceId, i.InvoiceDate, i.Total, c.FirstName, c.LastName\\nFROM Invoice i\\nJOIN Customer c ON c.CustomerId = i.CustomerId\\nWHERE c.FirstName = 'Frank' AND c.LastName = 'Harris'\\nORDER BY i.Total DESC\\nLIMIT 1;\"}"
tool - [{"InvoiceId":145,"InvoiceDate":"2010-09-23 00:00:00","Total":13.86,"FirstName":"Frank","LastName":"Harris"}]
ai - Tool calls: "convert_currency: {\"amount\":13.86,\"toCurrency\":\"EUR\"}"
tool - 13
ai - €13


## Tools with MCP

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `@langchain/mcp-adapters`.


In [7]:
import { MultiServerMCPClient } from "@langchain/mcp-adapters";

// Connect to the mcp-time server for timezone-aware operations
// This Go-based server provides tools for current time, relative time parsing,
// timezone conversion, duration arithmetic, and time comparison
const mcpClient = new MultiServerMCPClient({
    mcpServers: {
        time: {
            transport: "stdio",
            command: "npx",
            args: ["-y", "@theo.foobar/mcp-time"],
        }
    },
    useStandardContentBlocks: true,
});

// Load tools from the MCP server
const mcpTools = await mcpClient.getTools();
console.log(`Loaded ${mcpTools.length} MCP tools:`, mcpTools.map(t => t.name));


Loaded 5 MCP tools: [
  "add_time",
  "compare_time",
  "convert_timezone",
  "current_time",
  "relative_time"
]


Create an agent combining SQL queries, currency conversion, and timezone operations from MCP.


In [8]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agentWithMCP = createAgent({
    model: "openai:gpt-5",
    tools: [...mcpTools, executeSQL, convertCurrency],
    systemPrompt: SYSTEM,
    middleware: []
});


Test with a question requiring database queries, timezone conversion, and currency operations.


In [9]:
import { HumanMessage } from "langchain";

const complexQuestion = `
Find Frank Harris in the database. 
What is the current time in his country (USA/Eastern)?
How long ago was his most recent invoice from today?
Finally, show his total spending in EUR.
`;

const stream = await agentWithMCP.stream({
    messages: [new HumanMessage(complexQuestion)],
}, {
    streamMode: "values",
});

for await (const state of stream) {
    const message = state.messages.at(-1);
    const content = message.content || `Tool calls: ${JSON.stringify(message.tool_calls?.map(t => `${t.name}: ${JSON.stringify(t.args)}`).join(", "))}`;
    console.log(`${message.type} - ${content}`)
}

// Clean up the MCP connection
await mcpClient.close();


human - 
Find Frank Harris in the database. 
What is the current time in his country (USA/Eastern)?
How long ago was his most recent invoice from today?
Finally, show his total spending in EUR.

ai - Tool calls: "execute_sql: {\"query\":\"SELECT name FROM sqlite_master WHERE type='table' AND (name LIKE '%customer%' OR name LIKE '%client%' OR name LIKE '%invoice%' OR name LIKE '%user%' OR name LIKE '%person%') ORDER BY name LIMIT 5;\"}"
tool - [{"name":"Customer"},{"name":"Invoice"},{"name":"InvoiceLine"}]
ai - Tool calls: "execute_sql: {\"query\":\"SELECT CustomerId, FirstName, LastName, Country FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris' LIMIT 5;\"}"
tool - [{"CustomerId":16,"FirstName":"Frank","LastName":"Harris","Country":"USA"}]
ai - Tool calls: "execute_sql: {\"query\":\"SELECT MAX(InvoiceDate) AS LatestInvoiceDate,\\n       CAST(julianday('now') - julianday(MAX(InvoiceDate)) AS INTEGER) AS DaysSince\\nFROM Invoice\\nWHERE CustomerId = 16;\"}"
tool - [{"LatestI